# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 5 2022 8:35AM,241033,10,0085821067,ISDIN Corporation,Released
1,Jul 5 2022 8:35AM,241032,10,Eme-100686,Emerginnova,Released
2,Jul 5 2022 8:35AM,241032,10,Enova-10240,Emerginnova,Released
3,Jul 5 2022 8:35AM,241032,10,DNMD-22949,Emerginnova,Released
4,Jul 5 2022 8:28AM,241031,19,ADV80003767,"AdvaGen Pharma, Ltd",Released
5,Jul 5 2022 8:27AM,241030,19,8229464,"Innogenix, LLC",Released
6,Jul 5 2022 8:24AM,241029,10,0085829065,ISDIN Corporation,Released
7,Jul 5 2022 8:24AM,241029,10,0085828711,ISDIN Corporation,Released
8,Jul 5 2022 8:24AM,241029,10,0085829346,ISDIN Corporation,Released
9,Jul 5 2022 8:24AM,241029,10,0085829410,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
25,241029,Released,28
26,241030,Released,1
27,241031,Released,1
28,241032,Released,3
29,241033,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Released
id,
241029,28
241030,1
241031,1
241032,3
241033,1


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Released
id,
241002,55
241003,48
241004,59
241005,54
241006,55


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,Released
id,
241002,55
241003,48
241004,59
241005,54
241006,55


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Released
0,241002,55
1,241003,48
2,241004,59
3,241005,54
4,241006,55


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet
Column Executing does not exist yet


ShipmentStatus,id,Released
0,241002,55
1,241003,48
2,241004,59
3,241005,54
4,241006,55


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id          int64
Released    int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 5 2022 8:35AM,241033,10,ISDIN Corporation
1,Jul 5 2022 8:35AM,241032,10,Emerginnova
4,Jul 5 2022 8:28AM,241031,19,"AdvaGen Pharma, Ltd"
5,Jul 5 2022 8:27AM,241030,19,"Innogenix, LLC"
6,Jul 5 2022 8:24AM,241029,10,ISDIN Corporation
34,Jul 5 2022 8:20AM,241027,10,"Methapharm, Inc."
68,Jul 5 2022 8:20AM,241028,10,"Methapharm, Inc."
77,Jul 5 2022 8:15AM,241026,10,"Methapharm, Inc."
78,Jul 5 2022 8:08AM,241013,10,ISDIN Corporation
110,Jul 5 2022 8:07AM,241022,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Released
0,Jul 5 2022 8:35AM,241033,10,ISDIN Corporation,1
1,Jul 5 2022 8:35AM,241032,10,Emerginnova,3
2,Jul 5 2022 8:28AM,241031,19,"AdvaGen Pharma, Ltd",1
3,Jul 5 2022 8:27AM,241030,19,"Innogenix, LLC",1
4,Jul 5 2022 8:24AM,241029,10,ISDIN Corporation,28
5,Jul 5 2022 8:20AM,241027,10,"Methapharm, Inc.",34
6,Jul 5 2022 8:20AM,241028,10,"Methapharm, Inc.",9
7,Jul 5 2022 8:15AM,241026,10,"Methapharm, Inc.",1
8,Jul 5 2022 8:08AM,241013,10,ISDIN Corporation,32
9,Jul 5 2022 8:07AM,241022,10,ISDIN Corporation,60


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released
0,Jul 5 2022 8:35AM,241033,10,ISDIN Corporation,1
1,Jul 5 2022 8:35AM,241032,10,Emerginnova,3
2,Jul 5 2022 8:28AM,241031,19,"AdvaGen Pharma, Ltd",1
3,Jul 5 2022 8:27AM,241030,19,"Innogenix, LLC",1
4,Jul 5 2022 8:24AM,241029,10,ISDIN Corporation,28
5,Jul 5 2022 8:20AM,241027,10,"Methapharm, Inc.",34
6,Jul 5 2022 8:20AM,241028,10,"Methapharm, Inc.",9
7,Jul 5 2022 8:15AM,241026,10,"Methapharm, Inc.",1
8,Jul 5 2022 8:08AM,241013,10,ISDIN Corporation,32
9,Jul 5 2022 8:07AM,241022,10,ISDIN Corporation,60


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Jul 5 2022 8:35AM,241033,10,ISDIN Corporation,1
1,Jul 5 2022 8:35AM,241032,10,Emerginnova,3
2,Jul 5 2022 8:28AM,241031,19,"AdvaGen Pharma, Ltd",1
3,Jul 5 2022 8:27AM,241030,19,"Innogenix, LLC",1
4,Jul 5 2022 8:24AM,241029,10,ISDIN Corporation,28


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Executing does not exist yet
Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released
0,Jul 5 2022 8:35AM,241033,10,ISDIN Corporation,1
1,Jul 5 2022 8:35AM,241032,10,Emerginnova,3
2,Jul 5 2022 8:28AM,241031,19,"AdvaGen Pharma, Ltd",1
3,Jul 5 2022 8:27AM,241030,19,"Innogenix, LLC",1
4,Jul 5 2022 8:24AM,241029,10,ISDIN Corporation,28


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released
0,Jul 5 2022 8:35AM,241033,10,ISDIN Corporation,1
1,Jul 5 2022 8:35AM,241032,10,Emerginnova,3
2,Jul 5 2022 8:28AM,241031,19,"AdvaGen Pharma, Ltd",1
3,Jul 5 2022 8:27AM,241030,19,"Innogenix, LLC",1
4,Jul 5 2022 8:24AM,241029,10,ISDIN Corporation,28


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released
Warehouse,,
10,6748470,1288
19,482061,2


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released
0,10,6748470,1288
1,19,482061,2


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released
0,10,1288
1,19,2


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1288
1,19,Released,2


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,19
Status,,
Released,1288,2


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,19
0,Released,1288,2


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,19
0,Released,1288,2


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()